In [4]:
import pandas as pd

def transform_crypto_data_with_date_check(csv_path_or_df):
    # Load the CSV or use DataFrame directly
    if isinstance(csv_path_or_df, str):
        df = pd.read_csv(csv_path_or_df)
    else:
        df = csv_path_or_df.copy()

    # Rename and format
    df.rename(columns={
        'datetime': 'date',
        'symbol': 'tic',
        'open': 'open',
        'high': 'high',
        'low': 'low',
        'close': 'close',
        'volume': 'volume'
    }, inplace=True)

    # Convert to datetime and extract day of week
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.weekday  # Monday=0, Sunday=6
    df['date'] = df['date'].dt.date    # convert back to just date

    # Normalize ticker symbol
    df['tic'] = df['tic'].apply(lambda x: x.split(':')[-1].lower())

    # Reorder columns (no df.columns.name set)
    df = df[['date', 'close', 'high', 'low', 'open', 'volume', 'tic', 'day']]

    # Check for missing consecutive dates
    sorted_dates = pd.Series(pd.to_datetime(df['date'])).sort_values()
    full_range = pd.date_range(start=sorted_dates.min(), end=sorted_dates.max(), freq='D')
    missing_dates = full_range.difference(sorted_dates)

    if not missing_dates.empty:
        print("⚠️ Missing consecutive dates:")
        for d in missing_dates:
            print(f" - {d.date()}")
    else:
        print("✅ All dates are consecutive.")

    return df


In [13]:
df1 = transform_crypto_data_with_date_check("data/BTCUSD/BTCUSD_daily.csv")
df1.tail()

⚠️ Missing consecutive dates:
 - 2014-12-05
 - 2014-12-07
 - 2014-12-09
 - 2014-12-11
 - 2014-12-13
 - 2014-12-14
 - 2014-12-15
 - 2014-12-16
 - 2014-12-17
 - 2014-12-19
 - 2014-12-20
 - 2014-12-21
 - 2014-12-22
 - 2014-12-23
 - 2014-12-24
 - 2014-12-25
 - 2014-12-26
 - 2014-12-27
 - 2014-12-28
 - 2014-12-29
 - 2014-12-30
 - 2014-12-31
 - 2015-01-01
 - 2015-01-02
 - 2015-01-03
 - 2015-01-04
 - 2015-01-05
 - 2015-01-06
 - 2015-01-07
 - 2015-01-09
 - 2015-01-10
 - 2015-01-11
 - 2015-01-12
 - 2015-01-18


,date,close,high,low,open,volume,tic,day
3718,2025-03-10,78544.71,84033.66,77389.00,80708.21,21375.101210,btcusd,0
3719,2025-03-11,82914.51,83600.00,76555.00,78548.18,17096.379803,btcusd,1
3720,2025-03-12,83659.43,84442.69,80572.21,82911.81,12511.546352,btcusd,2
3721,2025-03-13,81073.43,84310.80,79890.00,83659.45,10504.151009,btcusd,3
3722,2025-03-14,81864.02,82238.51,80771.17,81071.50,1108.391470,btcusd,4


In [12]:
df2 = transform_crypto_data_with_date_check("data/ETHUSD/ETHUSD_daily.csv")
df2.tail()

✅ All dates are consecutive.


,date,close,high,low,open,volume,tic,day
3213,2025-03-10,1864.12,2152.48,1809.49,2020.02,362085.544760,ethusd,0
3214,2025-03-11,1923.08,1962.83,1754.13,1864.52,338544.564728,ethusd,1
3215,2025-03-12,1907.93,1956.15,1829.49,1923.02,227196.406122,ethusd,2
3216,2025-03-13,1863.81,1922.09,1820.86,1907.88,156769.604089,ethusd,3
3217,2025-03-14,1890.16,1899.32,1860.46,1863.80,14045.997199,ethusd,4


In [11]:
df3 = transform_crypto_data_with_date_check("data/SOLUSD/SOLUSD_daily.csv")
df3.tail()

✅ All dates are consecutive.


,date,close,high,low,open,volume,tic,day
1387,2025-04-04,122.74,124.81,113.26,117.14,2.374234e+06,solusd,4
1388,2025-04-05,120.25,122.81,115.81,122.74,8.066482e+05,solusd,5
1389,2025-04-06,105.82,121.05,103.71,120.25,2.141831e+06,solusd,6
1390,2025-04-07,106.94,113.05,95.16,105.81,5.120355e+06,solusd,0
1391,2025-04-08,110.61,112.57,106.63,106.93,1.412966e+06,solusd,1


In [8]:
merged_df = pd.concat([df1, df2, df3], ignore_index=True)
merged_df.sort_values(by=['tic', 'date'], inplace=True)
merged_df.head()

,date,close,high,low,open,volume,tic,day
0,2014-12-01,370.0,370.0,300.00,300.0,0.0,btcusd,0
1,2014-12-02,378.0,378.0,377.00,377.0,15.0,btcusd,1
2,2014-12-03,378.0,378.0,377.01,377.9,0.0,btcusd,2
3,2014-12-04,377.1,377.1,377.10,377.1,0.0,btcusd,3
4,2014-12-06,378.0,378.0,378.00,378.0,0.0,btcusd,5


In [9]:
def align_dataframes_on_common_start(*dfs):
    # Step 1: Find the latest of the three start dates
    start_dates = [df['date'].min() for df in dfs]
    common_start = max(start_dates)  # The latest among the start dates

    # Step 2: Filter all DataFrames to start from that date
    aligned_dfs = [df[df['date'] >= common_start].reset_index(drop=True) for df in dfs]

    print(f"✅ All DataFrames aligned from common start date: {common_start}")
    return aligned_dfs


In [10]:
df1_aligned, df2_aligned, df3_aligned = align_dataframes_on_common_start(df1, df2, df3)

merged_df = pd.concat([df1_aligned, df2_aligned, df3_aligned], ignore_index=True)
merged_df.sort_values(by=['date'], inplace=True)
merged_df.tail()


✅ All DataFrames aligned from common start date: 2021-06-17


,date,close,high,low,open,volume,tic,day
4121,2025-04-04,122.74,124.81,113.26,117.14,2.374234e+06,solusd,4
4122,2025-04-05,120.25,122.81,115.81,122.74,8.066482e+05,solusd,5
4123,2025-04-06,105.82,121.05,103.71,120.25,2.141831e+06,solusd,6
4124,2025-04-07,106.94,113.05,95.16,105.81,5.120355e+06,solusd,0
4125,2025-04-08,110.61,112.57,106.63,106.93,1.412966e+06,solusd,1


In [19]:
merged_df.to_csv("crypto_data.csv", index=False)

In [6]:
import pandas as pd
import os

def transform_crypto_data_with_date_check(csv_path_or_df, ticker):
    # Load the CSV or use DataFrame directly
    if isinstance(csv_path_or_df, str):
        df = pd.read_csv(csv_path_or_df)
    else:
        df = csv_path_or_df.copy()

    df.rename(columns={
        'datetime': 'date',
        'symbol': 'tic',
        'open': 'open',
        'high': 'high',
        'low': 'low',
        'close': 'close',
        'volume': 'volume'
    }, inplace=True)

    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.weekday
    df['date'] = df['date'].dt.date
    df['tic'] = ticker.lower()
    df = df[['date', 'close', 'high', 'low', 'open', 'volume', 'tic', 'day']]

    return df

def align_dataframes_on_common_range(*dfs):
    start_dates = [df['date'].min() for df in dfs]
    end_dates = [df['date'].max() for df in dfs]

    common_start = max(start_dates)
    common_end = min(end_dates)

    aligned_dfs = [
        df[(df['date'] >= common_start) & (df['date'] <= common_end)].reset_index(drop=True)
        for df in dfs
    ]

    print(f"All DataFrames aligned to date range: {common_start} → {common_end}")
    return aligned_dfs


def build_crypto_dataset(
    data_dir="data",
    tickers=["SPY", "VO", "VB", "AGG", "VNQ", "GLD", "BIL", "VWO"],
    interval="daily",
    start_date=None,
    end_date=None,
    output_csv="data.csv"
):
    all_dfs = []

    for ticker in tickers:
        file_path = os.path.join(data_dir, ticker, f"{ticker}_{interval}.csv")
        if not os.path.exists(file_path):
            print(f" File not found: {file_path}")
            continue

        df = transform_crypto_data_with_date_check(file_path, ticker)

        # Filter by date range if specified
        if start_date:
            df = df[df['date'] >= pd.to_datetime(start_date).date()]
        if end_date:
            df = df[df['date'] <= pd.to_datetime(end_date).date()]

        all_dfs.append(df)

    if len(all_dfs) < 2:
        print("Need at least 2 tickers with valid data.")
        return None

    aligned_dfs = align_dataframes_on_common_range(*all_dfs)
    merged_df = pd.concat(aligned_dfs, ignore_index=True)
    merged_df.sort_values(by=['date', 'tic'], inplace=True)
    merged_df.to_csv(output_csv, index=False)

    print(f"Final merged dataset saved to: {output_csv}")
    return merged_df


In [7]:
build_crypto_dataset(data_dir="newdata")

All DataFrames aligned to date range: 2007-05-30 → 2025-04-11
Final merged dataset saved to: data.csv


,date,close,high,low,open,volume,tic,day
13494,2007-05-30,99.38000,99.47000,99.27000,99.45000,977600.0,agg,2
26988,2007-05-30,91.60000,91.60000,91.50000,91.57998,1550.0,bil,2
22490,2007-05-30,64.72000,64.86000,64.60001,64.86000,2744800.0,gld,2
0,2007-05-30,153.48000,153.53999,151.34000,151.46001,131273376.0,spy,2
8996,2007-05-30,75.28999,75.32651,74.23000,74.36000,35300.0,vb,2
...,...,...,...,...,...,...,...,...
4497,2025-04-11,533.94000,536.43000,520.07000,523.01000,97866334.0,spy,4
13493,2025-04-11,205.82000,206.44000,199.28430,202.69000,1652602.0,vb,4
22489,2025-04-11,83.63000,83.80000,81.07000,82.05000,5276100.0,vnq,4
8995,2025-04-11,243.62000,244.33000,236.59260,239.25000,1396703.0,vo,4
